In [1]:
import sys
import scipy
import numpy as np
import pandas as pd
import matplotlib
import seaborn as sns

from pandas import ExcelFile
import matplotlib.pyplot as plt

# Cause plots to be displayed in the notebook:
%matplotlib inline
#print('Available plot styles: ' + str(plt.style.available))
plt.style.use('seaborn-whitegrid')

print('Python: \t', sys.version)
print('Scipy:  \t', scipy.__version__)
print('Numpy:  \t', np.__version__)
print('Pandas: \t', pd.__version__)
print('MatPlotLib: \t', matplotlib.__version__)
print('Author: \t Lawrance Koh (@LK)')
print('Team:   \t DS-Angels')

def mean_confidence_interval(data, confidence=0.95):
    a = 1.0 * np.array(data)
    n = len(a)
    m, se = np.mean(a), stats.sem(a)
    h = se * stats.t.ppf((1+confidence) / 2., n-1)
    return m, m-h, m+h


Python: 	 3.8.5 (default, Sep  3 2020, 21:29:08) [MSC v.1916 64 bit (AMD64)]
Scipy:  	 1.5.2
Numpy:  	 1.19.2
Pandas: 	 1.1.3
MatPlotLib: 	 3.3.2
Author: 	 Lawrance Koh (@LK)
Team:   	 DS-Angels


In [2]:
col_names = ["ReviewPro ID", "Hotel",
             "GRI_tot", "GRI_pos", "GRI_neu", "GRI_neg", "GRI_nr", 
             "BusinessCenter_tot", "BusinessCenter_pos", "BusinessCenter_neu", "BusinessCenter_neg", "BusinessCenter_nr",
             "Cleanliness_tot", "Cleanliness_pos", "Cleanliness_neu", "Cleanliness_neg", "Cleanliness_nr",
             "Decoration_tot", "Decoration_pos", "Decoration_neu", "Decoration_neg", "Decoration_nr",
             "Entertainment_tot", "Entertainment_pos", "Entertainment_neu", "Entertainment_neg", "Entertainment_nr",
             "Food_tot", "Food_pos", "Food_neu", "Food_neg", "Food_nr",
             "Location_tot", "Location_pos", "Location_neu", "Location_neg", "Location_nr",
             "Reception_tot", "Reception_pos", "Reception_neu", "Reception_neg", "Reception_nr",
             "Room_tot", "Room_pos", "Room_neu", "Room_neg", "Room_nr",
             "Service_tot", "Service_pos", "Service_neu", "Service_neg", "Service_nr",
             "Value_tot", "Value_pos", "Value_neu", "Value_neg", "Value_nr"]

data_df = pd.read_excel("ReviewProSentiment Data.xlsx", index_col="Hotel", skiprows=lambda x: x in [1,2], names=col_names)
ratings_df = pd.read_excel("ReviewProSentiment Data.xlsx", sheet_name="STB Rating", index_col="Hotel_Name")
#match, combine and add ratings column to dataframe 
#index joined on Hotel name
data_df["Star_Category"] = ratings_df["Star_Category"]

pd.options.display.max_columns = 57
data_df.head()


,ReviewPro ID,GRI_tot,GRI_pos,GRI_neu,GRI_neg,GRI_nr,BusinessCenter_tot,BusinessCenter_pos,BusinessCenter_neu,BusinessCenter_neg,BusinessCenter_nr,Cleanliness_tot,Cleanliness_pos,Cleanliness_neu,Cleanliness_neg,Cleanliness_nr,Decoration_tot,Decoration_pos,Decoration_neu,Decoration_neg,Decoration_nr,Entertainment_tot,Entertainment_pos,Entertainment_neu,Entertainment_neg,Entertainment_nr,Food_tot,Food_pos,Food_neu,Food_neg,Food_nr,Location_tot,Location_pos,Location_neu,Location_neg,Location_nr,Reception_tot,Reception_pos,Reception_neu,Reception_neg,Reception_nr,Room_tot,Room_pos,Room_neu,Room_neg,Room_nr,Service_tot,Service_pos,Service_neu,Service_neg,Service_nr,Value_tot,Value_pos,Value_neu,Value_neg,Value_nr,Star_Category
Hotel,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
8 on Claymore Serviced Residences,146727,56,53,1,2,0,2,0,0,2,0,41,40,1,0,0,NaN,NaN,NaN,NaN,NaN,0,0,0,0,0,0,0,0,0,0,41,41,0,0,0,2,0,0,2,0,41,38,2,1,0,41,39,0,2,0,41,38,1,2,0,5
Amara Hotel,121074,1352,769,465,118,0,20,0,0,20,0,163,104,36,23,0,2.0,1.0,1.0,0.0,0.0,0,0,0,0,0,0,0,0,0,0,135,98,32,5,0,20,0,0,20,0,168,102,50,16,0,168,92,44,32,0,133,53,51,29,0,4
Amara Sanctuary Resort Sentosa,175340,1404,638,515,251,0,74,0,0,74,0,361,228,67,66,0,4.0,2.0,0.0,2.0,0.0,0,0,0,0,0,4,2,1,1,0,312,228,55,29,0,76,1,1,74,0,250,134,65,51,0,369,237,64,68,0,196,72,56,68,0,5
Amoy by Far East Hospitality,144642,46,46,0,0,0,19,0,0,19,0,28,24,0,4,0,NaN,NaN,NaN,NaN,NaN,0,0,0,0,0,0,0,0,0,0,21,17,0,4,0,19,0,0,19,0,28,24,0,4,0,28,24,0,4,0,21,15,2,4,0,3
Aqueen Hotel Balestier,144676,504,160,268,76,0,8,0,0,8,0,63,49,12,2,0,NaN,NaN,NaN,NaN,NaN,0,0,0,0,0,1,0,1,0,0,58,30,21,7,0,16,4,3,9,0,61,31,24,6,0,63,47,14,2,0,53,38,12,3,0,3


In [3]:
#inspect keyword / sentiments criterias on grading hotels
sentiments_df = pd.read_excel("ReviewProSentiment Data.xlsx", sheet_name="Sentiment Mentions", header=2, index_col="Concept", skipfooter=11)
sentiments_df.head(10)


,Mentions,Positive,Negative,Related Concepts
Concept,,,,
room,80006,61327,18679,"room, bed...and 71 more"
hotel,50193,43738,6455,"room, bed...and 72 more"
staff,46592,43953,2639,"room, bed...and 65 more"
location,43272,41922,1350,"room, bed...and 69 more"
service,19152,17017,2135,"room, bed...and 64 more"
breakfast,16735,13473,3262,"room, bed...and 60 more"
bed,14550,12543,2007,"room, bed...and 58 more"
food,13737,11874,1863,"room, bed...and 58 more"
pool,12238,10555,1683,"room, bed...and 62 more"


### general ratings criteria in order of mentions
1. room
2. star_category (hotel)
3. service (staff, service)
4. location
5. food (breakfast, food)
6. bed
7. value


In [4]:
### Slice And Dice

hotel_stargroup = data_df.groupby(['Star_Category'])
hotel_stargroup["Star_Category"].count()

#plt.pie(hotel_stargroup["Star_Category"].count())
#plt.show()

Star_Category
3    69
4    81
5    40
Name: Star_Category, dtype: int64

In [5]:
## Observe total values correlation
#hotel_stargroup[["GRI_tot", "Room_tot", "Service_tot", "Location_tot", "Food_tot", "Value_tot"]].agg(["mean"]).transpose()
hotel_stargroup[["GRI_tot", "Room_tot", "Service_tot", "Location_tot", "Food_tot", "Value_tot"]].corr()


GRI_tot  Room_tot  Service_tot  Location_tot  \
Star_Category                                                               
3             GRI_tot       1.000000  0.869072     0.896115      0.877435   
              Room_tot      0.869072  1.000000     0.987750      0.979265   
              Service_tot   0.896115  0.987750     1.000000      0.992079   
              Location_tot  0.877435  0.979265     0.992079      1.000000   
              Food_tot      0.758547  0.869337     0.873347      0.874031   
              Value_tot     0.827177  0.985566     0.967079      0.978892   
4             GRI_tot       1.000000  0.822407     0.844486      0.792074   
              Room_tot      0.822407  1.000000     0.988600      0.976559   
              Service_tot   0.844486  0.988600     1.000000      0.990940   
              Location_tot  0.792074  0.976559     0.990940      1.000000   
              Food_tot      0.543282  0.668943     0.662058      0.669067   
              Value_tot     0.762182  0.989139     0.977345      0.984455   
5             GRI_tot       1.000000  0.870685     0.912218      0.916741   
              Room_tot      0.870685  1.000000     0.992212      0.986299   
              Service_tot   0.912218  0.992212     1.000000      0.997891   
              Location_tot  0.916741  0.986299     0.997891      1.000000   
              Food_tot      0.795450  0.968252     0.948805      0.938338   
              Value_tot     0.872053  0.997907     0.992036      0.989981   

                            Food_tot  Value_tot  
Star_Category                                    
3             GRI_tot       0.758547   0.827177  
              Room_tot      0.869337   0.985566  
              Service_tot   0.873347   0.967079  
              Location_tot  0.874031   0.978892  
              Food_tot      1.000000   0.859377  
              Value_tot     0.859377   1.000000  
4             GRI_tot       0.543282   0.762182  
              Room_tot      0.668943   0.989139  
              Service_tot   0.662058   0.977345  
              Location_tot  0.669067   0.984455  
              Food_tot      1.000000   0.662924  
              Value_tot     0.662924   1.000000  
5             GRI_tot       0.795450   0.872053  
              Room_tot      0.968252   0.997907  
              Service_tot   0.948805   0.992036  
              Location_tot  0.938338   0.989981  
              Food_tot      1.000000   0.960415  
              Value_tot     0.960415   1.000000

In [6]:
#define function for Pearson coefficient correlation testing

from scipy.stats import pearsonr
def sentiments_correlation_test(data1, data2, stars, sentiment, overall='GRI', p_value=0.05):
    stat, p = pearsonr(data1, data2)
    
    print('***Testing for %s star hotels; \'%s\' sentiments against \'%s\'' % (stars, sentiment, overall))
    print('stat=%.3f, p=%.3f' % (stat, p))
    if p > p_value:
        print('Probably independent\n')
    else:
        print('Probably dependent\n')
    
data = data_df[data_df["Star_Category"] == 3]
data1 = data["GRI_tot"]
data2 = data["Service_tot"]
sentiments_correlation_test(data1, data2, "3", "Service")

data2 = data["Room_tot"]
sentiments_correlation_test(data1, data2, "3", "Room")

data2 = data["Location_tot"]
sentiments_correlation_test(data1, data2, "3", "Location")

data2 = data["Value_tot"]
sentiments_correlation_test(data1, data2, "3", "Value")

data2 = data["Food_tot"]
sentiments_correlation_test(data1, data2, "3", "Food")


data = data_df[data_df["Star_Category"] == 4]
data1 = data["GRI_tot"]
data2 = data["Service_tot"]
sentiments_correlation_test(data1, data2, "4", "Service")

data2 = data["Room_tot"]
sentiments_correlation_test(data1, data2, "4", "Room")

data2 = data["Location_tot"]
sentiments_correlation_test(data1, data2, "4", "Location")

data2 = data["Value_tot"]
sentiments_correlation_test(data1, data2, "4", "Value")

data2 = data["Food_tot"]
sentiments_correlation_test(data1, data2, "4", "Food")


data = data_df[data_df["Star_Category"] == 5]
data1 = data["GRI_tot"]
data2 = data["Service_tot"]
sentiments_correlation_test(data1, data2, "5", "Service")

data2 = data["Room_tot"]
sentiments_correlation_test(data1, data2, "5", "Room")

data2 = data["Location_tot"]
sentiments_correlation_test(data1, data2, "5", "Location")

data2 = data["Value_tot"]
sentiments_correlation_test(data1, data2, "5", "Value")

data2 = data["Food_tot"]
sentiments_correlation_test(data1, data2, "5", "Food")


***Testing for 3 star hotels; 'Service' sentiments against 'GRI'
stat=0.896, p=0.000
Probably dependent

***Testing for 3 star hotels; 'Room' sentiments against 'GRI'
stat=0.869, p=0.000
Probably dependent

***Testing for 3 star hotels; 'Location' sentiments against 'GRI'
stat=0.877, p=0.000
Probably dependent

***Testing for 3 star hotels; 'Value' sentiments against 'GRI'
stat=0.827, p=0.000
Probably dependent

***Testing for 3 star hotels; 'Food' sentiments against 'GRI'
stat=0.759, p=0.000
Probably dependent

***Testing for 4 star hotels; 'Service' sentiments against 'GRI'
stat=0.844, p=0.000
Probably dependent

***Testing for 4 star hotels; 'Room' sentiments against 'GRI'
stat=0.822, p=0.000
Probably dependent

***Testing for 4 star hotels; 'Location' sentiments against 'GRI'
stat=0.792, p=0.000
Probably dependent

***Testing for 4 star hotels; 'Value' sentiments against 'GRI'
stat=0.762, p=0.000
Probably dependent

***Testing for 4 star hotels; 'Food' sentiments against 'GRI'
stat